In [1]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, \
                            precision_recall_curve, average_precision_score, roc_auc_score, roc_curve, auc, accuracy_score
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.utils.class_weight import compute_class_weight


from imblearn.over_sampling import RandomOverSampler


from matplotlib import pyplot as plt

import warnings

import numpy as np
import pandas as pd
from pathlib import Path
import shap

import xgboost
import lightgbm as lgb

import time

In [2]:
data_file_loc = Path('/disks/data/datasets/ADNI/')

In [3]:
raw_df = pd.read_csv(data_file_loc/'ADNI Data SPSS-CSV.csv')

In [4]:
col_names = list(raw_df.columns)
col_names

['GENDER',
 'maritalstatus',
 'APOE4',
 'ADASQ4_bl',
 'DX',
 'AGE',
 'FDG_bl',
 'CDRSB_bl',
 'ADAS11_bl',
 'ADAS13_bl',
 'MMSE_bl',
 'RAVLT_immediate_bl',
 'RAVLT_learning_bl',
 'RAVLT_forgetting_bl',
 'RAVLT_perc_forgetting_bl',
 'Ventricles_bl',
 'Hippocampus_bl',
 'WholeBrain_bl',
 'Entorhinal_bl',
 'Fusiform_bl',
 'MidTemp_bl']

In [5]:
# shufle 
df = raw_df.sample(frac=1).reset_index(drop=True)
df.head()

,GENDER,maritalstatus,APOE4,ADASQ4_bl,DX,AGE,FDG_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,...,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,Ventricles_bl,Hippocampus_bl,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl
0,Female,Married,0,8,MCI,78.6,1.318400,1.5,9.00,19.00,...,40.0,6.0,9.0,90.0000,52877.00000,5941.000000,1051620.000,3278.00000,16249.00000,18784.00000
1,Female,Widowed,0,4,MCI,83.7,1.228990,0.5,8.00,12.00,...,50.0,4.0,4.0,33.3333,30054.00000,6488.000000,905972.000,3491.00000,14554.00000,16551.00000
2,Female,Divorced,0,2,CN,68.5,1.173280,0.0,1.00,3.00,...,53.0,3.0,-1.0,-10.0000,51558.00000,6969.000000,1064940.000,3303.00000,18820.00000,21969.00000
3,Female,Married,0,7,CN,67.0,1.227286,0.0,10.67,19.67,...,28.0,4.0,4.0,50.0000,40274.12831,6794.012692,1020753.123,3485.98162,17303.26072,19450.01906
4,Male,Married,1,7,MCI,72.1,1.096340,0.5,15.33,22.33,...,27.0,4.0,7.0,100.0000,32799.00000,6566.000000,1077770.000,2898.00000,19822.00000,21282.00000


In [6]:
ros = RandomOverSampler(random_state=0)
df_resampled, y_resampled = ros.fit_resample(df, df.DX)

# shufle 
df_resampled = df_resampled.sample(frac=1).reset_index(drop=True)
df.head()

from collections import Counter
print(sorted(Counter(y_resampled).items()))

[('CN', 1050), ('Dementia', 1050), ('MCI', 1050)]


In [7]:
# convert categorical columns to one-hot encodings
df_ohe = pd.get_dummies(df_resampled, columns=['GENDER'])
df_ohe = pd.get_dummies(df_ohe, columns=['maritalstatus'])
# df_ohe.to_csv("df_ohe_1.csv")
df_ohe.head()

,APOE4,ADASQ4_bl,DX,AGE,FDG_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,...,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,GENDER_Female,GENDER_Male,maritalstatus_Divorced,maritalstatus_Married,maritalstatus_Never married,maritalstatus_Widowed
0,1,1,MCI,72.3,1.156400,1.5,9.00,11.00,30,44.0,...,871342.000,4119.00000,12430.00000,19215.00000,1,0,0,0,0,1
1,0,3,CN,67.7,1.227286,0.0,9.33,12.33,30,39.0,...,1020753.123,3485.98162,17303.26072,19450.01906,1,0,0,0,1,0
2,0,0,CN,65.1,1.227286,0.0,9.67,10.67,28,57.0,...,1020753.123,3485.98162,17303.26072,19450.01906,1,0,0,1,0,0
3,0,5,Dementia,62.7,1.350600,2.0,18.00,27.00,20,34.0,...,1012760.000,3592.00000,15906.00000,20834.00000,1,0,0,1,0,0
4,1,1,CN,69.2,1.227286,0.0,5.33,8.33,29,58.0,...,1020753.123,3485.98162,17303.26072,19450.01906,1,0,0,1,0,0


In [8]:
# list of the categorical and continuous columns
cat_names = ['GENDER','maritalstatus']

cont_names = col_names.copy()
cont_names.remove('DX')
cont_names.remove('GENDER')
cont_names.remove('maritalstatus')


In [9]:
# normalize the continuous columns
for cont_name in cont_names:
    df_ohe[cont_name] = (df_ohe[cont_name] - min(df_ohe[cont_name]) )/(max(df_ohe[cont_name]) - min(df_ohe[cont_name]))

# verify that columns are normalized
for cont_name in cont_names:
    print(max(df_ohe[cont_name]), min(df_ohe[cont_name]), np.mean(df_ohe[cont_name]))

1.0 0.0 0.2677777777777778
1.0 0.0 0.5528253968253971
1.0 0.0 0.5166578589944241
1.0 0.0 0.5088865716676555
1.0 0.0 0.19707936507936527
1.0 0.0 0.29255932503338433
1.0 0.0 0.34945482819647955
1.0 0.0 0.7397802197802159
1.0 0.0 0.48437793452054545
1.0 0.0 0.6016187167845409
1.0 0.0 0.7503860574516218
1.0 0.0 0.9211495273587036
1.0 0.0 0.25616977416223613
1.0 0.0 0.47146586246889555
1.0 0.0 0.421775030454858
1.0 0.0 0.4436539257796131
1.0 0.0 0.3864384272338237
1.0 0.0 0.42940292400822694


In [10]:
#df_ohe.to_csv("df_ohe_2.csv")
df_ohe.DX.value_counts()['CN'], df_ohe.DX.value_counts()['MCI'], df_ohe.DX.value_counts()['Dementia']


(1050, 1050, 1050)

In [11]:
sample_weights = compute_sample_weight(class_weight='balanced', y=df_ohe.DX)
class_weights = compute_class_weight(class_weight='balanced', classes = np.unique(df_ohe.DX), y=df_ohe.DX)

class_weights_dict_label = dict(zip(np.unique(df_ohe.DX),class_weights))
class_weights_dict_label_enc = dict(zip(range(3),class_weights))
class_weights

array([1., 1., 1.])

In [12]:
# datasets for binary classifications

# df_ohe_CN_Demen = df_ohe.loc[df['DX'].isin(['CN','Dementia'])]
# df_ohe_MCI_Demen = df_ohe.loc[df['DX'].isin(['MCI','Dementia'])]
# df_ohe_CN_MCI = df_ohe.loc[df['DX'].isin(['CN','MCI'])]
# # verify label-reassignment
# print(df_ohe_CN_Demen['DX'].unique(), df_ohe_MCI_Demen['DX'].unique(), df_ohe_CN_MCI['DX'].unique())

# # datasets for one vs all classifications
# df_ohe_CN_vs_all = df_ohe.copy()
# df_ohe_CN_vs_all = df_ohe_CN_vs_all.replace(['MCI','Dementia'],'Non')

# df_ohe_MCI_vs_all = df_ohe.copy()
# df_ohe_MCI_vs_all = df_ohe_MCI_vs_all.replace(['CN','Dementia'],'Non')

# df_ohe_Demen_vs_all = df_ohe.copy()
# df_ohe_Demen_vs_all = df_ohe_Demen_vs_all.replace(['CN','MCI'],'Non')

# # verify label-reassignment
# print(df_ohe_CN_vs_all['DX'].unique(), df_ohe_MCI_vs_all['DX'].unique(), df_ohe_Demen_vs_all['DX'].unique())

class Metrics:
    
    def __init__(self, benchmark_acc = None, benchmark_f1 = None, benchmark_spec = None, benchmark_sen = None, benchmark_auc = None, benchmark_mcc = None):
        self.b_accuracy = benchmark_acc
        self.b_f1 = benchmark_f1
        self.b_sensitivity = benchmark_sen
        self.b_specificity = benchmark_spec
        self.b_auc_roc = benchmark_auc
        self.b_mcc = benchmark_mcc
        
    def reset_history(self):
        self.accuracies = []
        self.f1s = []
        self.recalls = []
        self.precisions = []
        self.sensitivity = []
        self.specificity = []
        self.auc_roc = []        
        self.tp_tn_fp_fn = []
        self.mcc = []

        
    metrics_names = ('acc','mcc',)# 'specificity', 'sensitivity', 'f1_score', 'auc_roc', 'recall', 
                     #'precision', 'mcc',  'tp','fp','tn','fn')
    @classmethod
    def compute_mcc(cls, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        mcc = (tp*tn - fp*fn) / (np.sqrt(  (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)  ) + 1e-8) 
        return mcc
    
    @classmethod
    def compute_sensitivity(cls, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        sensitivity = tp/(tp+fn)
        return sensitivity
    
    @classmethod
    def compute_specificity(cls, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn/(fp+tn)
        return specificity
    
    @classmethod
    def compute_accuracy(cls, y_true, y_pred):
        accuracy = (y_true==y_pred).sum()/len(y_true)
        return accuracy
    
    def compute_metrics(self, y_true, y_pred, label_list, epoch, do_print = False, store_vals = False):
        accuracy = (y_true==y_pred).sum()/len(y_true)
        mcc = matthews_corrcoef(y_true, y_pred)
        
#         cm = confusion_matrix(y_true, y_pred, labels = label_list)

#         tn, fp, fn, tp = cm.ravel()

#         specificity = tn/(fp+tn)
#         sensitivity = tp/(tp+fn)
#         f1 = f1_score(y_true, y_pred)
#         recall = recall_score(y_true, y_pred)
#         precision = precision_score(y_true, y_pred)
#         auc_roc = roc_auc_score( y_true, y_pred )  
#         mcc = (tp*tn - fp*fn) / (np.sqrt(  (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)  ) + 1e-8) 
        
        metrics_vals = ( accuracy, mcc, )# specificity, sensitivity, f1, auc_roc, recall, precision, mcc, tp, fp, tn, fn )

        results = ( metrics_names, metrics_vals )        
        
        return results
    
    

In [13]:
# metrics_names = Metrics.metrics_names
# clf_metrics = [ i+'_'+j for j in metrics_names for i in classifier_namelist+NN_labels ]
# clf_metrics

In [14]:


# np.array([[1,2],[3,4]]).transpose().reshape(1,-1)
# tmp, inner_CV_results_df

In [15]:
NN_params = [ #{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': 0, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9, 'nesterovs_momentum': False, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9, 'nesterovs_momentum': True, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': 0, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9, 'nesterovs_momentum': True, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9, 'nesterovs_momentum': False, 'learning_rate_init': 0.2},
              {'solver': 'adam', 'learning_rate_init': 0.01},
              #{'solver': 'adam', 'learning_rate': 'invscaling', 'learning_rate_init': 0.01}
            ]


NN_labels = [ #"constant learning-rate", 
              #"constant with momentum",
              #"constant with Nesterov's momentum",
              #"inv-scaling learning-rate", 
              #"inv-scaling with momentum",
              #"inv-scaling with Nesterov's momentum", 
              "NN_Adam",
             # "NN_Adam_inv_scaling_LR"
            ]

NN_max_iter = 100

lgb_params  = {
          "objective" : "multiclass",
          "num_class" : 4,
          "num_leaves" : 60,
          "max_depth": -1,
          "learning_rate" : 0.01,
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "verbosity" : -1 }


classifier_namelist = [ "RF", "SVM", "LDA", "XGBoost", "LightGBM"]


# setup inner and outer k-fold 
skf_outer = StratifiedKFold(n_splits = 10)
skf_inner = StratifiedKFold(n_splits = 10)

# drop the target column 
X = (
        df_ohe
        .drop(['DX'], axis=1)
        .astype(float)
    )

# the target column
y = df_ohe.DX
label_list = sorted(y.unique()) 

# label encode the target variable
le = preprocessing.LabelEncoder()
le.fit(y)


y_label_enc = le.transform(y)
y_inv = le.inverse_transform(y_label_enc)

y_str = y
y = y_label_enc 

#le.classes_, (y_inv==y).unique()

metrics = Metrics()
metrics_names = Metrics.metrics_names

class_idx_to_name = dict(zip(range(len(le.classes_)),le.classes_))


np.unique(df_ohe.DX), le.classes_, label_list, (y_str==y_inv).unique(), class_idx_to_name



(array(['CN', 'Dementia', 'MCI'], dtype=object),
 array(['CN', 'Dementia', 'MCI'], dtype=object),
 ['CN', 'Dementia', 'MCI'],
 array([ True]),
 {0: 'CN', 1: 'Dementia', 2: 'MCI'})

In [16]:
np.any(np.isnan(X)), np.all(np.isfinite(X)), type(df_ohe)
#df_ohe.to_csv("df_ohe.csv")


(False, True, pandas.core.frame.DataFrame)

In [34]:
def save_shap_plots(model, X_test, class_idx_to_name):
    
    if (isinstance(model, RandomForestClassifier)):
        
        return 
    
        start_time_shap = time.time()
        explainer = shap.KernelExplainer(model.predict_proba, X_test)

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", module="shap")
            shap_values = explainer.shap_values(X_test, nsamples = 40) #nsamples='auto')

        end_time_shap = time.time()
        print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')
                
        f = plt.figure()
        shap.summary_plot(shap_values, X_test)
        f.savefig("figs/rf/summary_plot_rf.png", bbox_inches='tight', dpi=600)

        # plot the SHAP values for the Setosa output of the first instance
        shap.force_plot(explainer.expected_value[0], shap_values[0][0,:], X_test.iloc[0,:], link="logit")
        
        for class_idx, _ in enumerate(class_idx_to_name.keys()):
            #f = plt.figure()
            shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx], X_test)
            #f.savefig(f"figs/rf/force_plot_rf_{class_idx}_{class_idx_to_name[class_idx]}.png", bbox_inches='tight', dpi=600)

        for class_idx, _ in enumerate(class_idx_to_name.keys()):
            for colname in X_test.columns:
                #f = plt.figure()
                shap.dependence_plot(colname, shap_values[class_idx], X_test)   
                #f.savefig(f"figs/rf/dependence_plot_rf_{class_idx}_{class_idx_to_name[class_idx]}_col-{colname}.png", bbox_inches='tight', dpi=600)       
        
    elif (isinstance(model, svm.SVC)):
        
        return
    
        start_time_shap = time.time()
        explainer = shap.KernelExplainer(model.predict_proba, X_test, link="logit")
    
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", module="shap")
            shap_values = explainer.shap_values(X_test, nsamples = 30)

        end_time_shap = time.time()
        print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')

        f = plt.figure()
        shap.summary_plot(shap_values, X_test)
        f.savefig("figs/svm/summary_plot_svm.png", bbox_inches='tight', dpi=600)
        
        for class_idx in class_idx_to_name.keys():
            #f = plt.figure()
            shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx], X_test)
            #f.savefig(f"figs/svm/force_plot_svm_{class_idx}_{class_idx_to_name[class_idx]}.png", bbox_inches='tight', dpi=600)

        for class_idx in class_idx_to_name.keys():
            for colname in X_test.columns:
                #f = plt.figure()
                shap.dependence_plot(colname, shap_values[class_idx], X_test)   
                #f.savefig(f"figs/svm/dependence_plot_svm_{class_idx}_{class_idx_to_name[class_idx]}_col-{colname}.png", bbox_inches='tight', dpi=600)              
        
    elif (isinstance(model, LinearDiscriminantAnalysis)):
        return
    
        start_time_shap = time.time()
        explainer = shap.KernelExplainer(model.predict_proba, X_test, link="logit")
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", module="shap")
            shap_values = explainer.shap_values(X_test, nsamples=100)

        end_time_shap = time.time()
        print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')

        f = plt.figure()
        shap.summary_plot(shap_values, X_test)
        f.savefig("figs/lda/summary_plot_lda.png", bbox_inches='tight', dpi=600)    
        
        for class_idx in class_idx_to_name.keys():
            #f = plt.figure()
            shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx], X_test)
            #f.savefig(f"figs/lda/force_plot_lda_{class_idx}_{class_idx_to_name[class_idx]}.png", bbox_inches='tight', dpi=600)

        for class_idx in class_idx_to_name.keys():
            for colname in X_test.columns:
                #f = plt.figure()
                shap.dependence_plot(colname, shap_values[class_idx], X_test)   
                #f.savefig(f"figs/lda/dependence_plot_lda_{class_idx}_{class_idx_to_name[class_idx]}_col-{colname}.png", bbox_inches='tight', dpi=600)         
        
        
    elif (isinstance(model, xgboost.XGBClassifier)):
        return
    
        start_time_shap = time.time()
        explainer = shap.TreeExplainer(xgb)

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", module="shap")
            shap_values = explainer.shap_values(X_test)

        end_time_shap = time.time()
        print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')

        f = plt.figure()
        shap.summary_plot(shap_values, X_test)
        f.savefig("figs/xgboost/summary_plot_xgboost.png", bbox_inches='tight', dpi=600)  

        for class_idx in class_idx_to_name.keys():
            #f = plt.figure()
            shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx], X_test)
            #f.savefig(f"figs/xgboost/force_plot_xgboost_{class_idx}_{class_idx_to_name[class_idx]}.png", bbox_inches='tight', dpi=600)

        for class_idx in class_idx_to_name.keys():
            for colname in X_test.columns:
                #f = plt.figure()
                shap.dependence_plot(colname, shap_values[class_idx], X_test)   
                #f.savefig(f"figs/xgboost/dependence_plot_xgboost_{class_idx}_{class_idx_to_name[class_idx]}_col-{colname}.png", bbox_inches='tight', dpi=600)         
        
        
        
    elif (isinstance(model, lgb.LGBMClassifier)):
        return
    
        start_time_shap = time.time()
        explainer = shap.TreeExplainer(model)

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", module="shap")
            shap_values = explainer.shap_values(X_test)

        end_time_shap = time.time()
        print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')

        f = plt.figure()
        shap.summary_plot(shap_values, X_test)
        f.savefig("figs/lightGBM/summary_plot_lightGBM.png", bbox_inches='tight', dpi=600) 
        
        for class_idx in class_idx_to_name.keys():
            #f = plt.figure()
            shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx], X_test)
            #f.savefig(f"figs/lightGBM/force_plot_lightGBM_{class_idx}_{class_idx_to_name[class_idx]}.png", bbox_inches='tight', dpi=600)

        for class_idx in class_idx_to_name.keys():
            for colname in X_test.columns:
                #f = plt.figure()
                shap.dependence_plot(colname, shap_values[class_idx], X_test)   
                #f.savefig(f"figs/lightGBM/dependence_plot_lightGBM_{class_idx}_{class_idx_to_name[class_idx]}_col-{colname}.png", bbox_inches='tight', dpi=600)             
    
    elif (isinstance(model, MLPClassifier)):
        
        start_time_shap = time.time()
        explainer = shap.KernelExplainer(model.predict_proba, X_test)

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", module="shap")
            shap_values = explainer.shap_values(X_test, nsamples=80)

        end_time_shap = time.time()
        print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')

        f = plt.figure()
        shap.summary_plot(shap_values, X_test)
        f.savefig("figs/NN/summary_plot_NN.png", bbox_inches='tight', dpi=600) 
        
        for class_idx in class_idx_to_name.keys():
            #f = plt.figure()
            shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx], X_test)
            #f.savefig(f"figs/NN/force_plot_NN_{class_idx}_{class_idx_to_name[class_idx]}.png", bbox_inches='tight', dpi=600)

        for class_idx in class_idx_to_name.keys():
            for colname in X_test.columns:
                #f = plt.figure()
                shap.dependence_plot(colname, shap_values[class_idx], X_test)   
                #f.savefig(f"figs/NN/dependence_plot_NN_{class_idx}_{class_idx_to_name[class_idx]}_col-{colname}.png", bbox_inches='tight', dpi=600)             

        

In [40]:
pd.options.display.float_format = '{:,.6f}'.format

all_CV_results = []
all_CV_confusion_mats = []

clf_metrics_labels = [ i+'_'+j for j in metrics_names for i in classifier_namelist+NN_labels ]

outer_CV_results_df = pd.DataFrame({**dict.fromkeys(clf_metrics_labels, [])})
outer_CV_confusion_mats = []

show_header = True

outer_fold_idx = 0

# outer k-fold (for generalization performance estimation)
for train_idx_outer, test_idx_outer in skf_outer.split(X, y):

    outer_fold_idx += 1
    
    X_train_outer = X.iloc[train_idx_outer]
    y_train_outer = y[train_idx_outer]
    sample_weights_train_outer = sample_weights[train_idx_outer]
    
    X_test_outer = X.iloc[test_idx_outer]
    y_test_outer = y[test_idx_outer]  
    #class_weights_test_outer = sample_weights[test_idx_outer]
    
    inner_split_idx = 0
    
    # dataframe for holding the results from all the inner folds for all the candidate classifiers 
    inner_CV_results_df = pd.DataFrame({**dict.fromkeys(clf_metrics_labels, []) })
    inner_CV_fold_conf_mats = []
    
    start_time = time.time()
    
    # inner k-fold (for comparing model performance/hyper parameter tuning)
    for train_idx_inner, test_idx_inner in skf_inner.split(X_train_outer, y_train_outer):
        
        inner_split_idx +=1
        
        # inner fold's trainset
        X_train_inner = X_train_outer.iloc[train_idx_inner]
        y_train_inner = y_train_outer[train_idx_inner]
        sample_weights_train_inner = sample_weights_train_outer[train_idx_inner]


        # inner fold's testset
        X_test_inner = X_train_outer.iloc[test_idx_inner]
        y_test_inner = y_train_outer[test_idx_inner]
        #sample_weights_test_inner = sample_weights_train_outer[test_idx_inner]
        
        
        # initialize the classifiers
        rf_clf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)#, class_weight = class_weights_dict_label_enc)
        svm_clf = svm.SVC(kernel='rbf', probability=True)#, class_weight = class_weights_dict_label_enc)
        lda_clf = LinearDiscriminantAnalysis()
        xgb = xgboost.XGBClassifier()
        lgb_clf = lgb.LGBMClassifier(**lgb_params)
        
        # list of classifiers to iterate over (except for NN)
        classifiers = [rf_clf, svm_clf, lda_clf, xgb, lgb_clf ]

        # add the NNs to the list    
        for NN_config in NN_params:
            classifiers.append(MLPClassifier(alpha=1e-5, hidden_layer_sizes=(100, 75, 25, 16), random_state=1, **NN_config))
        
        # to hold the results from the classifiers in each inner fold        
        clf_results_inner_fold = []
        clf_conf_mats_inner_fold = []
        
        # iterate over the classifiers to compute their performance on a specific inner fold
        for clf in classifiers:
            
            #print(type(clf))
            
            #mean_acc = 0.0
            
            # some parameter combinations will not converge as can be seen on the
            # plots so they are ignored here
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", category=ConvergenceWarning,
                                        module="sklearn")
            
                # train the candidate classifier
                #if (isinstance(clf, xgboost.XGBClassifier)):
                #    clf.fit(X_train_inner, y_train_inner, sample_weight = sample_weights_train_inner)
                #else:
                clf.fit(X_train_inner, y_train_inner)

            #y_pred_probs = clf.predict_proba(X_test_inner)
            #y_pred_probs = clf.predict_proba(X_test.iloc[0:1])
            y_pred_inner = clf.predict(X_test_inner)

            # compute the mean accuracy
            # mean_acc = clf.score(X_test_inner, y_test_inner)

            _, results_metrics = metrics.compute_metrics(y_test_inner, y_pred_inner, label_list, -1)
            confusion_mat = confusion_matrix(y_test_inner, y_pred_inner)
            

            # store the result from the classifier for this inner fold
            clf_results_inner_fold.append(results_metrics)
            clf_conf_mats_inner_fold.append(confusion_mat)
        
        # create a tmp dataframe containing the results from different classifiers on this inner fold
        #tmp = pd.DataFrame([clf_results_inner_fold], columns=[*classifier_namelist, *NN_labels] )
        tmp = pd.DataFrame(np.array(clf_results_inner_fold).transpose().reshape(1,-1), columns=inner_CV_results_df.columns )        
        
        # update the results
        inner_CV_results_df = inner_CV_results_df.append(tmp)
        inner_CV_fold_conf_mats.append(clf_conf_mats_inner_fold)
        
    # store the result for the outer fold
    all_CV_results.append(inner_CV_results_df)       
    all_CV_confusion_mats.append(inner_CV_fold_conf_mats)
    
    
    # for re-training the classifiers on the outer fold's entire trainset
    # initialize the classifiers
    rf_clf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42, class_weight = class_weights_dict_label_enc)
    svm_clf = svm.SVC(kernel='rbf', probability=True, class_weight = class_weights_dict_label_enc)
    lda_clf = LinearDiscriminantAnalysis()
    xgb = xgboost.XGBClassifier()
    lgb_clf = lgb.LGBMClassifier(**lgb_params)

    # list of classifiers to iterate over (except for NN)
    classifiers = [rf_clf, svm_clf, lda_clf, xgb, lgb_clf ]

    # add the NNs to the list    
    for NN_config in NN_params:
        classifiers.append(MLPClassifier(alpha=1e-5, hidden_layer_sizes=(100, 75, 25, 16), random_state=1, **NN_config))
    
    # to hold the results from the classifiers in each outer fold        
    clf_results_outer_fold = []
    confusion_mats_outer_fold = []    
        
    for clf in classifiers:

        mean_acc = 0.0
            
        # some parameter combinations will not converge as can be seen on the
        # plots so they are ignored here
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning,
                                    module="sklearn")
            # train the candidate classifier
            #if (isinstance(clf, xgboost.XGBClassifier)):
            #    clf.fit(X_train_inner, y_train_inner, sample_weight = sample_weights_train_inner)
            #else:
            clf.fit(X_train_inner, y_train_inner)
            
            #y_pred_probs = clf.predict_proba(X_test_outer)
            #y_pred_probs = clf.predict_proba(X_test.iloc[0:1])
            y_pred_outer = clf.predict(X_test_outer)

            # compute the mean accuracy
            # mean_acc = clf.score(X_test_inner, y_test_inner)

            #print()
            _, results_metrics = metrics.compute_metrics(y_test_outer, y_pred_outer, label_list, -1)
            outer_confusion_mat = confusion_matrix(y_test_outer, y_pred_outer)
            
            if (outer_fold_idx == 1):
                print(type(clf))
                #save_shap_plots(clf, X_test_outer, class_idx_to_name)
                 
                    
            #print()

        # store the result from the classifier for this inner fold
        clf_results_outer_fold.append(results_metrics)
        confusion_mats_outer_fold.append(outer_confusion_mat)
    
                    
    #tmp = pd.DataFrame([clf_results_outer_fold], columns=[*classifier_namelist, *NN_labels] )
    tmp = pd.DataFrame(np.array(clf_results_outer_fold).transpose().reshape(1,-1), columns=outer_CV_results_df.columns )
    
    outer_CV_results_df = outer_CV_results_df.append(tmp)
    outer_CV_confusion_mats.append(confusion_mats_outer_fold)
    
    print(outer_CV_results_df.tail(1).to_string(index=False, header= show_header), end="\t")
    show_header = False

    #print(outer_CV_results_df.iloc[-1])
    end_time = time.time()
    print(f'Elapsed time: {end_time-start_time}')
    
    

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
<class 'sklearn.svm._classes.SVC'>
<class 'sklearn.discriminant_analysis.LinearDiscriminantAnalysis'>
<class 'xgboost.sklearn.XGBClassifier'>
<class 'lightgbm.sklearn.LGBMClassifier'>
<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
  RF_acc  SVM_acc  LDA_acc  XGBoost_acc  LightGBM_acc  NN_Adam_acc   RF_mcc  SVM_mcc  LDA_mcc  XGBoost_mcc  LightGBM_mcc  NN_Adam_mcc
0.930159 0.873016 0.819048     0.942857      0.926984     0.914286 0.897248 0.811440 0.729974     0.915158      0.891056     0.872392	Elapsed time: 26.23685884475708
0.926984 0.853968 0.815873 0.933333 0.885714 0.876190 0.892462 0.782278 0.723985 0.900340 0.828722 0.814323	Elapsed time: 24.89910364151001
0.946032 0.873016 0.819048 0.946032 0.917460 0.901587 0.920482 0.814127 0.730662 0.920398 0.876840 0.854918	Elapsed time: 24.96433115005493
0.955556 0.866667 0.806349 0.958730 0.917460 0.911111 0.934068 0.802772 0.709814 0.939047 0.876230 0.867546	

In [78]:
# Find which classifier performed best in each outer fold
for all_CV_result in all_CV_results:
    print(type(classifiers[np.argmax(all_CV_result.mean(axis = 0))]).__name__)

XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier


In [69]:
# confusion matrix of selected classifier averaged across outer folds 
selected_classifier_idx = 3
outer_CV_confusion_mats_np = np.array(outer_CV_confusion_mats)
print('outer_CV_confusion_mats_np.shape:',outer_CV_confusion_mats_np.shape)
conf_mat_freq_meaned = outer_CV_confusion_mats_np[:,selected_classifier_idx,:,:].mean(axis = 0)
print(conf_mat_freq_meaned)
conf_mat_pct_meaned = conf_mat_freq_meaned/conf_mat_freq_meaned.sum(axis =1)*100
print(conf_mat_pct_meaned)

outer_CV_confusion_mats_np.shape: (10, 6, 3, 3)
[[102.    0.    3. ]
 [  0.  103.4   1.6]
 [  3.    6.4  95.6]]
[[97.14285714  0.          2.85714286]
 [ 0.         98.47619048  1.52380952]
 [ 2.85714286  6.0952381  91.04761905]]


In [79]:
outer_CV_results_df.head(10)

,RF_acc,SVM_acc,LDA_acc,XGBoost_acc,LightGBM_acc,NN_Adam_acc,RF_mcc,SVM_mcc,LDA_mcc,XGBoost_mcc,LightGBM_mcc,NN_Adam_mcc
0,0.930159,0.873016,0.819048,0.942857,0.926984,0.914286,0.897248,0.811440,0.729974,0.915158,0.891056,0.872392
0,0.926984,0.853968,0.815873,0.933333,0.885714,0.876190,0.892462,0.782278,0.723985,0.900340,0.828722,0.814323
0,0.946032,0.873016,0.819048,0.946032,0.917460,0.901587,0.920482,0.814127,0.730662,0.920398,0.876840,0.854918
0,0.955556,0.866667,0.806349,0.958730,0.917460,0.911111,0.934068,0.802772,0.709814,0.939047,0.876230,0.867546
0,0.958730,0.885714,0.838095,0.968254,0.955556,0.936508,0.938450,0.830734,0.757223,0.952511,0.933517,0.905104
0,0.949206,0.892063,0.847619,0.958730,0.939683,0.933333,0.924537,0.841781,0.775673,0.938365,0.909565,0.900122
0,0.965079,0.892063,0.841270,0.968254,0.939683,0.926984,0.947805,0.840245,0.764091,0.952424,0.910571,0.890597
0,0.933333,0.869841,0.850794,0.952381,0.920635,0.933333,0.902473,0.809190,0.780247,0.930359,0.881366,0.900504
0,0.961905,0.882540,0.841270,0.961905,0.936508,0.933333,0.942985,0.824745,0.762782,0.943028,0.905803,0.900422
0,0.955556,0.885714,0.847619,0.965079,0.933333,0.917460,0.933771,0.831797,0.771989,0.947891,0.900122,0.877957


In [73]:
[print(inner_CV_result.mean(axis = 0)) for inner_CV_result in all_CV_results]

RF_acc         0.944985
SVM_acc        0.877619
LDA_acc        0.831063
XGBoost_acc    0.955555
LightGBM_acc   0.927353
NN_Adam_acc    0.917120
RF_mcc         0.918366
SVM_mcc        0.819566
LDA_mcc        0.747994
XGBoost_mcc    0.933729
LightGBM_mcc   0.891401
NN_Adam_mcc    0.876728
dtype: float64
RF_acc         0.949560
SVM_acc        0.881145
LDA_acc        0.838818
XGBoost_acc    0.956261
LightGBM_acc   0.933337
NN_Adam_acc    0.922396
RF_mcc         0.925210
SVM_mcc        0.824719
LDA_mcc        0.759811
XGBoost_mcc    0.934874
LightGBM_mcc   0.900497
NN_Adam_mcc    0.884872
dtype: float64
RF_acc         0.949570
SVM_acc        0.881145
LDA_acc        0.831766
XGBoost_acc    0.956619
LightGBM_acc   0.931930
NN_Adam_acc    0.917469
RF_mcc         0.925282
SVM_mcc        0.824457
LDA_mcc        0.749065
XGBoost_mcc    0.935410
LightGBM_mcc   0.898226
NN_Adam_mcc    0.876943
dtype: float64
RF_acc         0.944273
SVM_acc        0.880791
LDA_acc        0.837054
XGBoost_acc    0.95

[None, None, None, None, None, None, None, None, None, None]

In [70]:
# clf_results_inner_fold, ['Random Forest','SVM', 'LDA', *NN_labels], len(NN_params), len(classifiers)
outer_CV_results_df.mean(axis = 0)

RF_acc         0.948254
SVM_acc        0.877460
LDA_acc        0.832698
XGBoost_acc    0.955556
LightGBM_acc   0.927302
NN_Adam_acc    0.918413
RF_mcc         0.923428
SVM_mcc        0.818911
LDA_mcc        0.750644
XGBoost_mcc    0.933952
LightGBM_mcc   0.891379
NN_Adam_mcc    0.878389
dtype: float64